## Imports

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from torchvision import transforms
from torchvision.datasets import DatasetFolder
import cv2
import numpy as np

## Working with Data

In [ ]:
def readData(path):
    cap = cv2.VideoCapture(path)

    if (cap.isOpened()== False): 
        print("Error opening video stream or file")
        return None
    
    video = []
    
    while(cap.isOpened()):
        # Capture frame-by-frame
        ret, frame = cap.read()

        if ret == True:
            video.append(cv2.resize(frame, (120, 160)))
            
        # Break the loop
        else:
            break
 
    # When everything done, release the video capture object
    cap.release()

    return np.stack(video[:29], axis=0)

In [ ]:
data = DatasetFolder("./samples", loader=readData, extensions="avi")

In [ ]:
# video = data[0][0]

# print(type(video))
# for i in range(video.shape[0]):
#     cv2.imshow('Frame',video[i,:,:,:])

#     # Press Q on keyboard to  exit
#     if cv2.waitKey(25) & 0xFF == ord('q'):
#         break

# cv2.destroyAllWindows()

In [ ]:
X = np.stack([x[0] for x in data], axis=0)
Y = np.stack([x[1] for x in data], axis=0)

In [ ]:
X[0].shape